In [2]:
import tensorflow as tf
import time
from datetime import timedelta
import numpy as np
ks=tf.keras

from kgcnn.graph.base import GraphDict
from kgcnn.data.base import MemoryGraphList, MemoryGraphDataset
from kgcnn.data.qm import QMDataset
from kgcnn.layers.conv.acsf_conv import ACSFG2, ACSFG4

In [3]:
dataset = QMDataset(data_directory="ExampleQM", file_name="ThiolDisulfidExchange.csv", dataset_name="ThiolDisulfidExchange")
dataset.prepare_data(overwrite=True, make_sdf = True);
dataset.read_in_memory(label_column_name="energy");
dataset.map_list(method= "set_range", max_distance= 10.0, max_neighbours= 10000);
dataset.map_list(method="set_angle");
dataset.save();

ERROR:kgcnn.data.ThiolDisulfidExchange:Data directory does not exist.
ERROR:kgcnn.data.ThiolDisulfidExchange:Data directory does not exist.


FileNotFoundError: Can not find '.csv' table path 'None'.

In [9]:
dataset = MemoryGraphDataset(data_directory="ExampleQM", dataset_name="ThiolDisulfidExchange")
dataset.load();
dataset[0].keys()

INFO:kgcnn.data.ThiolDisulfidExchange:Load pickled dataset...


dict_keys(['node_symbol', 'node_number', 'node_coordinates', 'edge_indices', 'edge_number', 'graph_labels', 'range_indices', 'range_attributes', 'angle_indices', 'angle_indices_nodes', 'angle_attributes'])

In [4]:
elemental_mapping = [1,6,16]

# Radial parameters
cutoff_rad = 10
Rs_array   = [0.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
eta_array  = [0.0, 0.03, 0.08, 0.16, 0.3, 0.5]
rad_params = [(eta, Rs, cutoff_rad) for Rs in Rs_array for eta in eta_array]
rad_params = np.broadcast_to(rad_params, (len(elemental_mapping),len(rad_params),3))
print(rad_params.shape)


(3, 42, 3)


In [11]:
inputs = dataset.tensor([
    {"shape": (None,), "name": "node_number", "dtype": "int64", "ragged": True},
    {"shape": (None, 3), "name": "node_coordinates", "dtype": "float32", "ragged": True},
    {"shape": (None, 2), "name": "range_indices", "dtype": "int64", "ragged": True}
    ])
inputs = [x[:100] for x in inputs]
len(inputs), inputs[0].shape, inputs[1].shape, inputs[2].shape

(3,
 TensorShape([100, None]),
 TensorShape([100, None, 3]),
 TensorShape([100, None, 2]))

In [12]:
rad_symm_layer = ACSFG2(rad_params,elemental_mapping)
rad_symms = rad_symm_layer(inputs)

In [13]:
np.set_printoptions(precision=2)
print(rad_symms.to_tensor().shape)
print(rad_params[0])
for i in range(1):
    print(rad_symms[0,i].numpy())

(100, 15, 126)
[[ 0.    0.   10.  ]
 [ 0.03  0.   10.  ]
 [ 0.08  0.   10.  ]
 [ 0.16  0.   10.  ]
 [ 0.3   0.   10.  ]
 [ 0.5   0.   10.  ]
 [ 0.    1.5  10.  ]
 [ 0.03  1.5  10.  ]
 [ 0.08  1.5  10.  ]
 [ 0.16  1.5  10.  ]
 [ 0.3   1.5  10.  ]
 [ 0.5   1.5  10.  ]
 [ 0.    2.   10.  ]
 [ 0.03  2.   10.  ]
 [ 0.08  2.   10.  ]
 [ 0.16  2.   10.  ]
 [ 0.3   2.   10.  ]
 [ 0.5   2.   10.  ]
 [ 0.    2.5  10.  ]
 [ 0.03  2.5  10.  ]
 [ 0.08  2.5  10.  ]
 [ 0.16  2.5  10.  ]
 [ 0.3   2.5  10.  ]
 [ 0.5   2.5  10.  ]
 [ 0.    3.   10.  ]
 [ 0.03  3.   10.  ]
 [ 0.08  3.   10.  ]
 [ 0.16  3.   10.  ]
 [ 0.3   3.   10.  ]
 [ 0.5   3.   10.  ]
 [ 0.    3.5  10.  ]
 [ 0.03  3.5  10.  ]
 [ 0.08  3.5  10.  ]
 [ 0.16  3.5  10.  ]
 [ 0.3   3.5  10.  ]
 [ 0.5   3.5  10.  ]
 [ 0.    4.   10.  ]
 [ 0.03  4.   10.  ]
 [ 0.08  4.   10.  ]
 [ 0.16  4.   10.  ]
 [ 0.3   4.   10.  ]
 [ 0.5   4.   10.  ]]
[7.01e+00 5.51e+00 4.03e+00 2.92e+00 2.14e+00 1.62e+00 7.01e+00 6.41e+00
 5.61e+00 4.72e+00 3.82e+00 3

In [17]:
ang_inputs = dataset.tensor([
    {"shape": (None,), "name": "node_number", "dtype": "int64", "ragged": True},
    {"shape": (None, 3), "name": "node_coordinates", "dtype": "float32", "ragged": True},
    {"shape": (None, 3), "name": "angle_indices_nodes", "dtype": "int64", "ragged": True}
    ])
ang_inputs = [x[:100] for x in ang_inputs]

In [18]:
# Angular parameters
cutoff_ang    = 6
lambd_array   = [-1, 1]
zeta_array    = [1, 2, 4, 8, 16]
eta_ang_array = eta_array
ang_params    = [[eta, zeta, lambd, cutoff_ang] for eta in eta_ang_array for zeta in zeta_array for lambd in lambd_array]
ang_params = np.broadcast_to(ang_params, (int(len(elemental_mapping)*(len(elemental_mapping)+1)/2),len(ang_params),4))

print(ang_params.shape)

(6, 60, 4)


In [19]:
ang_symm_layer = ACSFG4(
    eta_zeta_lambda_rc=ang_params,
    element_mapping=elemental_mapping,
    element_pair_mapping=None,  # Automatic
    keep_pair_order=False,
    multiplicity=2.0
)
ang_symms = ang_symm_layer(ang_inputs)

In [20]:
print(ang_symms.to_tensor().shape)
print(ang_params[0])
for i in range(1):
    print(ang_symms[0,i].numpy())

(100, 15, 360)
[[ 0.    1.   -1.    6.  ]
 [ 0.    1.    1.    6.  ]
 [ 0.    2.   -1.    6.  ]
 [ 0.    2.    1.    6.  ]
 [ 0.    4.   -1.    6.  ]
 [ 0.    4.    1.    6.  ]
 [ 0.    8.   -1.    6.  ]
 [ 0.    8.    1.    6.  ]
 [ 0.   16.   -1.    6.  ]
 [ 0.   16.    1.    6.  ]
 [ 0.03  1.   -1.    6.  ]
 [ 0.03  1.    1.    6.  ]
 [ 0.03  2.   -1.    6.  ]
 [ 0.03  2.    1.    6.  ]
 [ 0.03  4.   -1.    6.  ]
 [ 0.03  4.    1.    6.  ]
 [ 0.03  8.   -1.    6.  ]
 [ 0.03  8.    1.    6.  ]
 [ 0.03 16.   -1.    6.  ]
 [ 0.03 16.    1.    6.  ]
 [ 0.08  1.   -1.    6.  ]
 [ 0.08  1.    1.    6.  ]
 [ 0.08  2.   -1.    6.  ]
 [ 0.08  2.    1.    6.  ]
 [ 0.08  4.   -1.    6.  ]
 [ 0.08  4.    1.    6.  ]
 [ 0.08  8.   -1.    6.  ]
 [ 0.08  8.    1.    6.  ]
 [ 0.08 16.   -1.    6.  ]
 [ 0.08 16.    1.    6.  ]
 [ 0.16  1.   -1.    6.  ]
 [ 0.16  1.    1.    6.  ]
 [ 0.16  2.   -1.    6.  ]
 [ 0.16  2.    1.    6.  ]
 [ 0.16  4.   -1.    6.  ]
 [ 0.16  4.    1.    6.  ]
 [ 0.16  8.  